## Import Data 

In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import time
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
# from imblearn.over_sampling import SMOTE
# from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTE, ADASYN

from sklearn import svm
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics

from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

In [ ]:
def feature_mat(df):
    X_mat = []
    for i in df.index:
        rowData = df.loc[i,:].values[0:10].tolist()
        X_mat.append(rowData)
    return np.array(X_mat)

df = pd.read_csv('processed_data.csv', names = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'y_i'])
X = feature_mat(df)
y = df['y_i']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20)

# df = pd.read_csv('processed_data.csv', names = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'y_i'])
# dumb_df = df[['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']]
# DAYs_df = pd.DataFrame()
# DAYs_df['TMAX'] = dumb_df.max(axis = 1)
# DAYs_df['TMIN'] = dumb_df.min(axis = 1)
# DAYs_df['TMEAN'] = dumb_df.mean(axis = 1)
# DAYs_df['TVAR'] = dumb_df.var(axis = 1)

# # Raw Feature & Labels
# X = DAYs_df.values
# y = df['y_i']

# # Built-in Random SPLIT
# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.80, test_size=0.20)

# Oversampling imbalanced minority class
# oversample = RandomOverSampler(sampling_strategy = 'minority')
# X_over, y_over = oversample.fit_resample(X, y)
# X_over, y_over = BorderlineSMOTE().fit_resample(X, y)
X_over, y_over = SMOTE().fit_resample(X_train, y_train)
X_over_test, y_over_test = SMOTE().fit_resample(X_test, y_test)

# For plotting decision boundary
# X_1_plt = df['D1']
# X_2_plt = df['D2']

In [ ]:
df.head()

In [ ]:
n_class_0 = np.size(df.index[df['y_i'] == 0].tolist())
n_class_1 = np.size(df.index[df['y_i'] == 1].tolist())
print(n_class_0)
print(n_class_1)
print(n_class_1*11)

In [ ]:
# Oversampling train-data
import random 

def Oversampling(X_train, y_train):
    n_class_0 = y_train.to_list().count(0)
    N = len(y_train.to_list())
    n_classes = 11
    X_temp = []
    y_temp = []

    for k in range(n_classes):
        n_new_example = n_class_0-y_train.to_list().count(k+1)
        index_arr = np.linspace(0,N-1,N, dtype=int)
        new_example_index_list = np.random.choice(index_arr, size=n_new_example, replace=True)
        
        for i in new_example_index_list:
            y_temp.append(y_train.to_list()[i])
            X_temp.append(X_train[i])

    X_over = np.concatenate((np.array(X_temp), X_train))
    y_over = pd.Series(y_temp+y_train.to_list())
    
    return X_over, y_over

In [ ]:
# X_over, y_over = Oversampling(X_train, y_train)
print(np.size(X_over), np.size(y_over), np.size(X_over_test), np.size(y_over_test))

## Model fitting and evaluation

In [ ]:
# 有个大问题，我好想不应该用 SVC，得用 OneVsOneClassifier。。。。。因为三种跑法得到的 support vectors 数量一样


In [ ]:
# SVC Default use OVO method for classification -- Reasons for choosing OVO is explained in essay
time_cost = []

# fit the model with no wieghts/penalizations
start = time.time()
clf_no_weights = svm.SVC(kernel='rbf', gamma='auto', class_weight=None).fit(X_train, y_train)
end = time.time()
time_cost.append(round(end-start,4))

# fit the model with proportional weights for minority class
start = time.time()
clf_weights = svm.SVC(kernel='rbf', gamma='auto', class_weight='balanced').fit(X_train, y_train)
                                                # Set the parameter C of class i to class_weight[i]*C for SVC. 
                                                # The “balanced” mode uses the values of y to automatically adjust weights 
                                                # inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))
end = time.time()
time_cost.append(round(end-start,4))


# fit the model with oversampled training data
start = time.time()
clf_oversample = svm.SVC(kernel='rbf', gamma='auto', class_weight=None).fit(X_over, y_over)
                                                # oversample the data for training have equivalent effect as changing 
                                                # class_weight proportionally 
end = time.time()
time_cost.append(round(end-start,4))


# Present Runtime of each
print(time_cost)


In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# print(scaler.fit(X_train))

# # The standardization only applies for OnevsOne otherwise will cause failure of convergence
# X_train_onevsone = scaler.transform(X_train)
# X_train_test = scaler.transform(X_test)
# print(X_train_onevsone)

In [ ]:
# clf = OneVsOneClassifier(LinearSVC(random_state=0, max_iter = 100000)).fit(X_train_onevsone, y_train)
# clf

In [ ]:
# one_vs_one_predict = clf.predict(X_train_test)
# print(metrics.classification_report(y_test, one_vs_one_predict, zero_division=1, digits=6))

In [ ]:
# No actions
clf_SVM_pred = clf_no_weights.predict(X_test)
# Use different misclassification penalties per class (C_i)
wclf_SVM_pred = clf_weights.predict(X_test)
# Oversampling training data with imbalance package
oclf_SVM_pred = clf_oversample.predict(X_test)

print("\n____________________Unweighted multi-SVM_______________________\n")
print(metrics.classification_report(y_test, clf_SVM_pred, zero_division=1, digits=6))
print("\n______________Proportionally Weighted multi-SVM________________\n")
print(metrics.classification_report(y_test, wclf_SVM_pred, digits=6))
print("\n______________Oversample train-data multi-SVM__________________\n")
print(metrics.classification_report(y_test, oclf_SVM_pred, zero_division=1, digits=6))

In [ ]:
print("\nclf")
print("SVM Accuracy Score:\t", metrics.accuracy_score(clf_SVM_pred, y_test)*100)
print("SVM f1 score:\t\t", metrics.f1_score(clf_SVM_pred, y_test, average="macro"))
print("SVM precision score:\t", metrics.precision_score(clf_SVM_pred, y_test, average="macro"))
print("SVM recall score:\t", metrics.recall_score(clf_SVM_pred, y_test, average="macro", zero_division=1))

print("\nwclf")
print("SVM Accuracy Score:\t", metrics.accuracy_score(wclf_SVM_pred, y_test)*100)
print("SVM f1 score:\t\t", metrics.f1_score(wclf_SVM_pred, y_test, average="macro"))
print("SVM precision score:\t", metrics.precision_score(wclf_SVM_pred, y_test, average="macro"))
print("SVM recall score:\t", metrics.recall_score(wclf_SVM_pred, y_test, average="macro", zero_division=1))

print("\noclf")
print("SVM Accuracy Score:\t", metrics.accuracy_score(oclf_SVM_pred, y_test)*100)
print("SVM f1 score:\t\t", metrics.f1_score(oclf_SVM_pred, y_test, average="macro"))
print("SVM precision score:\t", metrics.precision_score(oclf_SVM_pred, y_test, average="macro"))
print("SVM recall score:\t", metrics.recall_score(oclf_SVM_pred, y_test, average="macro", zero_division=1))
print("\n")

# print("\onevsone")
# print("SVM Accuracy Score:\t", metrics.accuracy_score(one_vs_one_predict, y_test)*100)
# print("SVM f1 score:\t\t", metrics.f1_score(one_vs_one_predict, y_test, average="macro"))
# print("SVM precision score:\t", metrics.precision_score(one_vs_one_predict, y_test, average="macro"))
# print("SVM recall score:\t", metrics.recall_score(one_vs_one_predict, y_test, average="macro", zero_division=1))
# print("\n")

In [ ]:
# SVMs = [clf_SVM_pred, wclf_SVM_pred, oclf_SVM_pred]
# dic = {}
# dic.keys() = ["Accuracy", "F1 Score", "Precision", "Recall"]
# for i in SVMs:
#    dic[keys[0]] = metrics.accuracy_score(SVMs[i], y_test)*100
#    dic[keys[1]] = metrics.f1_score(SVMs[i], y_test, average="macro")
#    dic[keys[2]] = metrics.precision_score(SVMs[i], y_test, average="macro")
#    dic[keys[3]] = metrics.recall_score(SVMs[i], y_test, average="macro", zero_division=1)
# print(dic)

## Confusion Matrix 

In [ ]:
# clf_no_weights
C = metrics.confusion_matrix(y_test, clf_SVM_pred)
sns.heatmap(pd.DataFrame(C), annot=True, vmin=-1, vmax=30, cmap="YlGnBu")

In [ ]:
# clf_weights
C_weight = metrics.confusion_matrix(y_test, wclf_SVM_pred)
sns.heatmap(pd.DataFrame(C_weight), annot=True, vmin=-1, vmax=30, cmap="YlGnBu")

In [ ]:
# clf_oversample
C_over = metrics.confusion_matrix(y_test, oclf_SVM_pred)
sns.heatmap(pd.DataFrame(C_over),annot=True, vmin=-1, vmax=30, cmap="YlGnBu")

## PR-curve

In [ ]:
# from sklearn.multiclass import OneVsOneClassifier
# from sklearn.svm import LinearSVC
# clf = OneVsOneClassifier(LinearSVC(random_state=0, max_iter=1e15)).fit(X_train, y_train)
# clf.predict(X_test)

In [ ]:
# # precision recall curve
# from sklearn.metrics import precision_recall_curve

# precision = dict()
# recall = dict()
# n_classes = 11
# for i in range(n_classes):
#     precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
#                                                         y_score[:, i])
#     plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))
    
# plt.xlabel("recall")
# plt.ylabel("precision")
# plt.legend(loc="best")
# plt.title("precision vs. recall curve")
# plt.show()

In [ ]:
y_score = clf_weights.decision_function(X_test)
print(y_score[20:30])
print(y_test[20:30])

In [ ]:
import numpy as np

y_test_lst = y_test.tolist()
print(y_test_lst[0:10])

new_y_test = []
for i in range(len(y_test_lst)):
    tmp = [0 for i in range(11)]
    num = y_test_lst[i]
    tmp[num] = 1
    new_y_test.append(tmp)

new_y_test = np.asarray(new_y_test, dtype=int)
print(new_y_test[0:10])

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

precision = dict()
recall = dict()
average_precision = dict()
for i in range(11):
#     print(y_test)
#     print(len(y_score), len(y_score[0]))
    precision[i], recall[i], _ = precision_recall_curve(new_y_test[:, i], y_score[:, i])
    average_precision[i] = average_precision_score(new_y_test[:, i], y_score[:, i])
    
precision["micro"], recall["micro"], _ = precision_recall_curve(new_y_test.ravel(), y_score.ravel())
average_precision["micro"] = average_precision_score(new_y_test, y_score, average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["micro"]))

In [ ]:
plt.figure()
plt.step(recall['micro'], precision['micro'], where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f} clf_weights'
    .format(average_precision["micro"]))

In [ ]:
y_score = clf_no_weights.decision_function(X_test)
print(y_score[20:30])
print(y_test[20:30])

In [ ]:
import numpy as np

y_test_lst = y_test.tolist()
print(y_test_lst[0:10])

new_y_test = []
for i in range(len(y_test_lst)):
    tmp = [0 for i in range(11)]
    num = y_test_lst[i]
    tmp[num] = 1
    new_y_test.append(tmp)

new_y_test = np.asarray(new_y_test, dtype=int)
print(new_y_test[0:10])

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

precision = dict()
recall = dict()
average_precision = dict()
for i in range(11):
#     print(y_test)
#     print(len(y_score), len(y_score[0]))
    precision[i], recall[i], _ = precision_recall_curve(new_y_test[:, i], y_score[:, i])
    average_precision[i] = average_precision_score(new_y_test[:, i], y_score[:, i])
    
precision["micro"], recall["micro"], _ = precision_recall_curve(new_y_test.ravel(), y_score.ravel())
average_precision["micro"] = average_precision_score(new_y_test, y_score, average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["micro"]))

In [ ]:
plt.figure()
plt.step(recall['micro'], precision['micro'], where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f} clf_no_weights'
    .format(average_precision["micro"]))

In [ ]:
y_score = clf_oversample.decision_function(X_test)
print(y_score[20:30])
print(y_test[20:30])

In [ ]:
import numpy as np

y_test_lst = y_test.tolist()
print(y_test_lst[0:10])

new_y_test = []
for i in range(len(y_test_lst)):
    tmp = [0 for i in range(11)]
    num = y_test_lst[i]
    tmp[num] = 1
    new_y_test.append(tmp)

new_y_test = np.asarray(new_y_test, dtype=int)
print(new_y_test[0:10])

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

precision = dict()
recall = dict()
average_precision = dict()
for i in range(11):
#     print(y_test)
#     print(len(y_score), len(y_score[0]))
    precision[i], recall[i], _ = precision_recall_curve(new_y_test[:, i], y_score[:, i])
    average_precision[i] = average_precision_score(new_y_test[:, i], y_score[:, i])
    
precision["micro"], recall["micro"], _ = precision_recall_curve(new_y_test.ravel(), y_score.ravel())
average_precision["micro"] = average_precision_score(new_y_test, y_score, average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["micro"]))

In [ ]:
plt.figure()
plt.step(recall['micro'], precision['micro'], where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f} clf_oversample'
    .format(average_precision["micro"]))

## ROC Curves of Multi-class

$\space$

## Contour map of decision boundary 